In [23]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
from collections import OrderedDict
import random
import torch.nn.functional as F
import torch.nn.functional as func
import collections
from sklearn.model_selection import train_test_split
from collections import Counter
from config import *

In [24]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # 解决由于多次加载 OpenMP 相关动态库而引起的冲突

In [25]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec 16 17:33:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:21:00.0 Off |                  Off |
| 30%   26C    P8              14W / 350W |   2553MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [26]:
# MobileNetV2（比lenet更复杂的CNN网络）网络中的线性瓶颈结构，原文中用于CIFAR-100任务
class LinearBottleNeck(nn.Module):

    def __init__(self, in_channels, out_channels, stride, t=6, class_num=100):
        super().__init__()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels, in_channels * t, 1),
            nn.BatchNorm2d(in_channels * t),
            nn.ReLU6(inplace=True),

            nn.Conv2d(in_channels * t, in_channels * t, 3, stride=stride, padding=1, groups=in_channels * t),
            nn.BatchNorm2d(in_channels * t),
            nn.ReLU6(inplace=True),

            nn.Conv2d(in_channels * t, out_channels, 1),
            nn.BatchNorm2d(out_channels)
        )

        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels

    def forward(self, x):

        residual = self.residual(x)

        if self.stride == 1 and self.in_channels == self.out_channels:
            residual += x

        return residual

class MobileNetV2(nn.Module):

    def __init__(self, class_num=20):
        super().__init__()

        self.pre = nn.Sequential(
            nn.Conv2d(3, 32, 1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True)
        )

        self.stage1 = LinearBottleNeck(32, 16, 1, 1)
        self.stage2 = self._make_stage(2, 16, 24, 2, 6)
        self.stage3 = self._make_stage(3, 24, 32, 2, 6)
        self.stage4 = self._make_stage(4, 32, 64, 2, 6)
        self.stage5 = self._make_stage(3, 64, 96, 1, 6)
        self.stage6 = self._make_stage(3, 96, 160, 1, 6)
        self.stage7 = LinearBottleNeck(160, 320, 1, 6)

        self.conv1 = nn.Sequential(
            nn.Conv2d(320, 1280, 1),
            nn.BatchNorm2d(1280),
            nn.ReLU6(inplace=True)
        )

        self.conv2 = nn.Conv2d(1280, class_num, 1)

    def forward(self, x):
        x = self.pre(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.stage5(x)
        x = self.stage6(x)
        x = self.stage7(x)
        x = self.conv1(x)
        x = F.adaptive_avg_pool2d(x, 1)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)

        return x

    def _make_stage(self, repeat, in_channels, out_channels, stride, t):

        layers = []
        layers.append(LinearBottleNeck(in_channels, out_channels, stride, t))

        while repeat - 1:
            layers.append(LinearBottleNeck(out_channels, out_channels, 1, t))
            repeat -= 1

        return nn.Sequential(*layers)

def mobilenetv2():
    return MobileNetV2()

In [27]:
def test_inference(model, test):
    """ Returns the test accuracy and loss.
    """
    tensor_x = torch.Tensor(test[0]).to(device)
    tensor_y = torch.Tensor(test[1]).to(device)
    test_dataset = TensorDataset(tensor_x, tensor_y)

    model.eval()
    loss, total, correct = 0.0, 0.0, 0.0

    criterion = nn.CrossEntropyLoss()
    testloader = DataLoader(test_dataset, batch_size=128,
                            shuffle=True)

    for batch_idx, (images, labels) in enumerate(testloader):
        with torch.no_grad():  # 在测试过程中不需要计算梯度，节省内存和加速计算
        # Inference
            outputs = model(images)
            batch_loss = criterion(outputs, labels.long())
            loss += batch_loss.item() * labels.size(0) # 计算损失值，更好反映模型输出概率分布与真实标签的差距

        # Prediction
            _, pred_labels = torch.max(outputs, 1)
            pred_labels = pred_labels.view(-1)
            correct += torch.sum(torch.eq(pred_labels, labels)).item()
            total += len(labels)
    #print(correct,"/",total)
    accuracy = correct/total
    return accuracy, loss

In [28]:
# 将CIFAR-100的100个类别转为20个类别（粒度更粗，降低任务复杂度）
def sparse2coarse(targets):
    """Convert Pytorch CIFAR100 sparse targets to coarse targets.

    Usage:
        trainset = torchvision.datasets.CIFAR100(path)
        trainset.targets = sparse2coarse(trainset.targets)
    """
    coarse_labels = np.array([ 4,  1, 14,  8,  0,  6,  7,  7, 18,  3,
                               3, 14,  9, 18,  7, 11,  3,  9,  7, 11,
                               6, 11,  5, 10,  7,  6, 13, 15,  3, 15,
                               0, 11,  1, 10, 12, 14, 16,  9, 11,  5,
                               5, 19,  8,  8, 15, 13, 14, 17, 18, 10,
                               16, 4, 17,  4,  2,  0, 17,  4, 18, 17,
                               10, 3,  2, 12, 12, 16, 12,  1,  9, 19,
                               2, 10,  0,  1, 16, 12,  9, 13, 15, 13,
                              16, 19,  2,  4,  6, 19,  5,  5,  8, 19,
                              18,  1,  2, 15,  6,  0, 17,  8, 14, 13])
    return coarse_labels[targets]

In [29]:
# 共有6w个图像，其中5w训练，1w测试
def CIFAR100():
    '''Return Cifar100
    '''
    train_dataset = torchvision.datasets.CIFAR100(root='../data/CIFAR-100',
                                            train=True,
                                            transform=transforms.ToTensor(),
                                            download=True)
    test_dataset = torchvision.datasets.CIFAR100(root='../data/CIFAR-100',
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)
    total_img,total_label = [],[]
    for imgs,labels in train_dataset:
        total_img.append(imgs.numpy())
        total_label.append(labels)
    for imgs,labels in test_dataset:
        total_img.append(imgs.numpy())
        total_label.append(labels) 
    total_img = np.array(total_img)
    total_label = np.array(sparse2coarse(total_label))

    cifar = [total_img, total_label]
    return cifar


In [30]:
# 基于 Dirichlet 分布 来模拟non-IID。返回一个形状为 (client_num, class_num) 的概率矩阵，每一行代表一个客户端对各类别的概率分布。
def get_prob(non_iid, client_num, class_num = 20):
    return np.random.dirichlet(np.repeat(non_iid, class_num), client_num)

In [31]:
def create_data(prob, size_per_client, dataset, N=20):
    total_each_class = size_per_client * np.sum(prob, 0)
    data, label = dataset


    # 为每个类别随机采样数据
    all_class_set = []
    for i in range(N):
        size = total_each_class[i]
        sub_data = data[label == i]
        sub_label = label[label == i]

        rand_indx = np.random.choice(len(sub_data), size=int(size), replace=False).astype(int)
        sub2_data, sub2_label = sub_data[rand_indx], sub_label[rand_indx]
        all_class_set.append((sub2_data, sub2_label))

    index = [0] * N
    clients, test = [], []

    for m in range(prob.shape[0]):  # 遍历客户端
        labels, images = [], []  # 训练数据
        tlabels, timages = [], [] # 测试数据

        # TODO_241216：这里每个client的测试集和它的训练集分布相同，并且最后测试时，也是计算所有client中的准确率的平均值
        # TODO_241216：别的FL方法也是这样做的吗？我也要这样做吗？
        for n in range(N):
            # 80%用于训练，20%用于测试
            # 这里的int向下取整，会导致实际的数据量比计算略小
            start, end = index[n], index[n] + int(prob[m][n] * size_per_client * 0.8)
            test_start, test_end = end, index[n] + int(prob[m][n] * size_per_client)

            image, label = all_class_set[n][0][start:end], all_class_set[n][1][start:end]
            test_image, test_label = all_class_set[n][0][test_start:test_end], all_class_set[n][1][test_start:test_end]

            # 记录当前类别的数据分配进度
            index[n] += int(prob[m][n] * size_per_client)

            labels.extend(label)
            images.extend(image)

            tlabels.extend(test_label)
            timages.extend(test_image)

        clients.append((np.array(images), np.array(labels)))
        test.append((np.array(timages), np.array(tlabels)))

    return clients, test

In [32]:

# 合并所有客户端的测试数据 （上面讲测试数据分成了不同的客户端）
# 但并没有使用，用途不明
def comb_client_test_func(client_test_data):
    comb_client_test_image = []
    comb_client_test_label = []
    for i in range(client_num):
        comb_client_test_image.extend(list(client_test_data[i][0]))
        comb_client_test_label.extend(list(client_test_data[i][1]))
    
    # 将测试图片和标签合并为 numpy 数组
    comb_client_test_image = np.array(comb_client_test_image)
    comb_client_test_label = np.array(comb_client_test_label)
    
    label_count = Counter(comb_client_test_label)
    print("测试集类别分布：")
    for label, count in sorted(label_count.items()):
        print(f"类别 {label}: {count} 个样本")
    
    return [comb_client_test_image, comb_client_test_label]

In [33]:
# 从数据集中按类别均匀抽取子集，并按照指定的比例 percentage 进行缩减，同时对数据进行随机打乱
def select_subset(whole_set, percentage):
    a = whole_set[0]
    b = whole_set[1]
    if len(a) != len(b):
        raise ValueError("Both arrays should have the same length.")

    if not 0 <= percentage <= 1:
        raise ValueError("Percentage must be between 0 and 1.")

    unique_classes = np.unique(b)

    a_prime = []
    b_prime = []

    for cls in unique_classes:
        indices = np.where(b == cls)[0]
        subset_size = int(len(indices) * percentage)

        selected_indices = np.random.choice(indices, subset_size, replace=False)

        a_prime.extend(a[selected_indices])
        b_prime.extend(b[selected_indices])

    a_prime, b_prime = np.array(a_prime), np.array(b_prime)

    # Shuffle arrays to randomize the order of elements
    shuffle_indices = np.random.permutation(len(a_prime))
    a_prime, b_prime = a_prime[shuffle_indices], b_prime[shuffle_indices]

    return [a_prime, b_prime]

In [34]:

# 准备数据集
# 这部分是我加的
cifar = CIFAR100()
prob = get_prob(non_iid, client_num, class_num=20)
client_data, client_test_data = create_data(prob, size_per_client, cifar, N=20)


all_images = []
all_labels = []
for data in client_data:
    all_images.extend(data[0])
    all_labels.extend(data[1])
comb_client_data = [np.array(all_images), np.array(all_labels)]

# 打印每个客户端训练数据情况
for i, (imgs, lbls) in enumerate(client_data):
    lbls = np.array(lbls)
    total_count = len(lbls)
    unique_classes, counts = np.unique(lbls, return_counts=True)
    # 创建一个长度为20的数组记录各类别计数，默认0
    class_counts = [0]*20
    for cls, cnt in zip(unique_classes, counts):
        class_counts[cls] = cnt
    # 打印格式：Client i: 总数 类别0计数 类别1计数 ... 类别19计数
    print("Client {}: {}".format(i, " ".join([str(total_count)] + [str(c) for c in class_counts])))
    

# 打印每个客户端测试数据情况
for i, (imgs, lbls) in enumerate(client_test_data):
    lbls = np.array(lbls)
    total_count = len(lbls)
    unique_classes, counts = np.unique(lbls, return_counts=True)
    class_counts = [0]*20
    for cls, cnt in zip(unique_classes, counts):
        class_counts[cls] = cnt
    # 打印格式：Client i Test: 总数 类别0计数 类别1计数 ... 类别19计数
    print("Client {} Test: {}".format(i, " ".join([str(total_count)] + [str(c) for c in class_counts])))


Files already downloaded and verified
Files already downloaded and verified
Client 0: 150 41 0 10 15 7 0 0 5 0 4 0 30 13 0 0 2 12 0 2 9
Client 1: 153 18 0 4 14 7 0 4 1 0 7 0 0 9 40 0 0 2 11 5 31
Client 2: 151 6 6 19 20 23 2 6 0 10 14 1 7 3 5 6 3 13 1 5 1
Client 3: 152 5 8 29 2 25 0 7 1 1 0 0 10 6 20 0 11 6 18 2 1
Client 4: 155 0 6 20 0 1 20 15 13 0 1 8 25 32 1 0 0 2 0 11 0
Client 5: 150 17 7 33 9 3 15 7 1 0 1 21 0 10 3 1 11 3 8 0 0
Client 6: 153 4 1 13 12 21 2 4 9 35 1 0 16 11 0 16 0 5 1 0 2
Client 7: 150 1 1 0 3 0 16 3 2 3 3 5 0 36 3 1 3 14 33 15 8
Client 8: 153 8 7 1 0 6 1 39 23 12 0 25 9 1 1 2 0 7 1 9 1
Client 9: 148 1 10 0 10 6 16 0 24 2 0 0 13 0 35 21 6 0 2 0 2
Client 10: 151 14 0 1 0 0 58 0 3 4 0 3 11 6 7 0 2 19 0 9 14
Client 11: 150 20 3 4 11 13 12 0 9 0 1 6 4 0 0 2 37 9 3 8 8
Client 12: 149 0 0 0 0 1 27 4 0 50 0 2 5 20 0 1 22 1 11 2 3
Client 13: 148 1 28 0 8 3 1 4 0 5 0 23 45 0 5 2 7 0 7 9 0
Client 14: 152 3 5 0 0 25 23 0 4 1 0 0 6 20 0 17 0 0 1 16 31
Client 15: 153 1 45 0 1 5 

In [35]:
# 本地训练并更新权重，返回更新后的模型权重、平均训练损失以及第一个迭代的梯度信息
def update_weights(model_weight, dataset, learning_rate, local_epoch):
    model = mobilenetv2().to(device)
    model.load_state_dict(model_weight)

    model.train()
    epoch_loss = []
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    Tensor_set = TensorDataset(torch.Tensor(dataset[0]).to(device), torch.Tensor(dataset[1]).to(device))
    data_loader = DataLoader(Tensor_set, batch_size=128, shuffle=True)

    first_iter_gradient = None  # 初始化变量来保存第一个iter的梯度

    for iter in range(local_epoch):
        batch_loss = []
        for batch_idx, (images, labels) in enumerate(data_loader):
            model.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            batch_loss.append(loss.item()/images.shape[0])

            # 保存第一个iter的梯度
            if iter == 0 and batch_idx == 0:
                first_iter_gradient = {}
                for name, param in model.named_parameters():
                    first_iter_gradient[name] = param.grad.clone()
                # 保存 BatchNorm 层的 running mean 和 running variance
                for name, module in model.named_modules():
                    if isinstance(module, nn.BatchNorm2d):
                        first_iter_gradient[name + '.running_mean'] = module.running_mean.clone()
                        first_iter_gradient[name + '.running_var'] = module.running_var.clone()

        epoch_loss.append(sum(batch_loss)/len(batch_loss))

    return model.state_dict(), sum(epoch_loss) / len(epoch_loss), first_iter_gradient

In [36]:
# 计算模型权重的差异，并根据学习率 lr 对权重差异进行缩放
def weight_differences(n_w, p_w, lr):
    w_diff = copy.deepcopy(n_w)
    for key in w_diff.keys():
        if 'num_batches_tracked' in key:
            continue
        w_diff[key] = (p_w[key] - n_w[key]) * lr
    return w_diff

In [37]:
# 也是本地训练，不过引入了本文的权重修正机制
def update_weights_correction(model_weight, dataset, learning_rate, local_epoch, c_i, c_s):
    model = mobilenetv2().to(device)
    model.load_state_dict(model_weight)

    model.train()
    epoch_loss = []
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    Tensor_set = TensorDataset(torch.Tensor(dataset[0]).to(device), torch.Tensor(dataset[1]).to(device))
    data_loader = DataLoader(Tensor_set, batch_size=200, shuffle=True)

    for iter in range(local_epoch):
        batch_loss = []
        for batch_idx, (images, labels) in enumerate(data_loader):
            model.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            batch_loss.append(loss.sum().item()/images.shape[0])
        epoch_loss.append(sum(batch_loss)/len(batch_loss))
        corrected_graident = weight_differences(c_i, c_s, learning_rate)
        orginal_model_weight = model.state_dict()
        corrected_model_weight = weight_differences(corrected_graident, orginal_model_weight, 1)  # 这里缩放权重为1
        model.load_state_dict(corrected_model_weight)

    return model.state_dict(),  sum(epoch_loss) / len(epoch_loss)

In [38]:
def average_weights(w):
    """
    Returns the average of the weights.
    """
    w_avg = copy.deepcopy(w[0])
    for key in w_avg.keys():
        if 'num_batches_tracked' in key:
            continue
        for i in range(1, len(w)):
            w_avg[key] += w[i][key]
        w_avg[key] = torch.div(w_avg[key], len(w))
    return w_avg

In [39]:
# baseline: server-only
def server_only(initial_w, global_round, gamma, E):
    test_model = mobilenetv2().to(device)
    train_w = copy.deepcopy(initial_w)
    test_acc = []
    train_loss = []
    for round in tqdm(range(global_round)):
        # if gamma > 0.001:
        #     gamma = gamma * 0.99
        # Server side local training
        # 从comb中（论文中说明为全部训练数据）选择固定比率的server数据（并且是保证类别均衡的）
        server_data = select_subset(comb_client_data, server_percentage)
        update_server_w, round_loss, _ = update_weights(train_w, server_data, gamma, E)
        train_w = update_server_w
        test_model.load_state_dict(train_w)
        train_loss.append(round_loss)
        # Test Accuracy
        test_a = 0
        for i in client_test_data:
            ac = test_inference(test_model,i)[0]
            test_a = test_a + ac
        test_a = test_a/len(client_test_data)
        test_acc.append(test_a)
        # print(test_a)
    return test_acc, train_loss

In [40]:
def fedavg(initial_w, global_round, eta, K, M):
    test_model = mobilenetv2().to(device)
    train_w = copy.deepcopy(initial_w)
    test_acc = []
    train_loss = []
    for round in tqdm(range(global_round)):
        local_weights, local_loss = [], []
        # Client side local training
        # if eta > 0.001:
        #     eta = eta * 0.99
        sampled_client = random.sample(range(client_num), M)
        for i in sampled_client:
            update_client_w, client_round_loss, _ = update_weights(train_w, client_data[i], eta, K)
            local_weights.append(update_client_w)
            local_loss.append(client_round_loss)

        train_w = average_weights(local_weights)

        # Test Accuracy
        test_model.load_state_dict(train_w)
        loss_avg = sum(local_loss)/ len(local_loss)
        train_loss.append(loss_avg)
        test_a = 0
        for i in client_test_data:
            ac = test_inference(test_model,i)[0]
            test_a = test_a + ac
        test_a = test_a/len(client_test_data)
        test_acc.append(test_a)
#         print(test_a)
    return test_acc, train_loss


In [41]:
def CLG_SGD(initial_w, global_round, eta, gamma, K, E, M):
    test_model = mobilenetv2().to(device)
    train_w = copy.deepcopy(initial_w)
    test_acc = []
    train_loss = []
    for round in tqdm(range(global_round)):
        # 学习率衰减，这里默认注释掉了
        # if eta > 0.001:
        #     eta = eta * 0.99
        # if gamma > 0.001:
        #     gamma = gamma * 0.99
        local_weights, local_loss = [], []
        # Client side local training
        # 从总共client_num客户端中选择M个训练
        sampled_client = random.sample(range(client_num), M)
        for i in sampled_client:
            update_client_w, client_round_loss, _ = update_weights(train_w, client_data[i], eta, K)
            local_weights.append(update_client_w)
            local_loss.append(client_round_loss)
        train_w = average_weights(local_weights)
        # Server side local training
        # 从comb中（论文中说明为全部训练数据）选择固定比率的server数据（并且是保证类别均衡的）
        # TODO_241216:这里是每一轮都重新选择数据，我的场景中可以这样吗？
        server_data = select_subset(comb_client_data, server_percentage)
        
        
        # 第一轮都打印服务器数据情况，简化输出格式
        if round == 0:
            s_imgs, s_lbls = server_data
            s_lbls = np.array(s_lbls)
            total_count = len(s_lbls)
            unique_classes, counts = np.unique(s_lbls, return_counts=True)
            class_counts = [0]*20
            for cls, cnt in zip(unique_classes, counts):
                class_counts[cls] = cnt

            # 输出格式: Server round: 总数 类别0计数 类别1计数 ... 类别19计数
            print("Server {}: {}".format(round, " ".join([str(total_count)] + [str(c) for c in class_counts])))

        update_server_w, round_loss, _ = update_weights(train_w, server_data, gamma, E)
        train_w = update_server_w
        local_loss.append(round_loss)

        # Test Accuracy
        test_model.load_state_dict(train_w)
        loss_avg = sum(local_loss)/ len(local_loss)
        train_loss.append(loss_avg)   # 计算所有客户端和服务器一起的平均损失

        test_a = 0
        # 遍历客户端测试数据，计算平均准确率
        for i in client_test_data:
            ac = test_inference(test_model,i)[0]
            test_a = test_a + ac
        test_a = test_a/len(client_test_data)
        test_acc.append(test_a)
#         print(test_a)
    return test_acc, train_loss

In [42]:
def Fed_C(initial_w, global_round, eta, gamma, K, E, M):
    test_model = mobilenetv2().to(device)
    train_w = copy.deepcopy(initial_w)
    test_acc = []
    train_loss = []
    for round in tqdm(range(global_round)):
        # if eta > 0.001:
        #     eta = eta * 0.99
        # if gamma > 0.001:
        #     gamma = gamma * 0.99
        local_weights, local_loss = [], []
        g_i_list = []
        server_data = select_subset(comb_client_data, server_percentage)
        # 计算Server gradient
        _, _, g_s = update_weights(train_w, server_data, gamma, 1)

        # 计算Client gradient
        sampled_client = random.sample(range(client_num), M)
        for i in sampled_client:
            _, _, g_i = update_weights(train_w, client_data[i], eta, 1)
            g_i_list.append(g_i)


        # Client side local training
        for i in range(len(sampled_client)):
            update_client_w, client_round_loss = update_weights_correction(train_w, client_data[sampled_client[i]], eta, K, g_i_list[i], g_s)
            local_weights.append(update_client_w)
            local_loss.append(client_round_loss)
        train_w = average_weights(local_weights)
        # Server side local training
        update_server_w, round_loss, _ = update_weights(train_w, server_data, gamma, E)
        train_w = update_server_w
        local_loss.append(round_loss)

        # Test Accuracy
        test_model.load_state_dict(train_w)
        loss_avg = sum(local_loss)/ len(local_loss)
        train_loss.append(loss_avg)

        test_a = 0
        for i in client_test_data:
            ac = test_inference(test_model,i)[0]
            test_a = test_a + ac
        test_a = test_a/len(client_test_data)
        test_acc.append(test_a)
    return test_acc, train_loss


In [43]:
def Fed_S(initial_w, global_round, eta, gamma, K, E, M):
    test_model = mobilenetv2().to(device)
    train_w = copy.deepcopy(initial_w)
    test_acc = []
    train_loss = []
    for round in tqdm(range(global_round)):
        # if eta > 0.001:
        #     eta = eta * 0.99
        # if gamma > 0.001:
        #     gamma = gamma * 0.99
        local_weights, local_loss = [], []
        g_i_list = []
        # Server gradient
        server_data = select_subset(comb_client_data, server_percentage)
        _, _, g_s = update_weights(train_w, server_data, gamma, 1)

        # Client gradient
        sampled_client = random.sample(range(client_num), M)
        for i in sampled_client:
            _, _, g_i = update_weights(train_w, client_data[i], eta, 1)
            g_i_list.append(g_i)


        # Client side local training
        for i in range(len(sampled_client)):
            update_client_w, client_round_loss, _ = update_weights(train_w, client_data[sampled_client[i]], eta, K)
            local_weights.append(update_client_w)
            local_loss.append(client_round_loss)
        train_w = average_weights(local_weights)

        # Server aggregation correction
        g_i_average = average_weights(g_i_list)
        correction_g = weight_differences(g_i_average, g_s, K*eta)
        train_w = weight_differences(correction_g, copy.deepcopy(train_w), 1)


        # Server side local training
        update_server_w, round_loss, _ = update_weights(train_w, server_data, gamma, E)
        train_w = update_server_w
        local_loss.append(round_loss)

        # Test Accuracy
        test_model.load_state_dict(train_w)
        loss_avg = sum(local_loss)/ len(local_loss)
        train_loss.append(loss_avg)

        test_a = 0
        for i in client_test_data:
            ac = test_inference(test_model,i)[0]
            test_a = test_a + ac
        test_a = test_a/len(client_test_data)
        test_acc.append(test_a)
    return test_acc, train_loss

In [44]:
# 初始化模型与参数
# 这部分是我补充的
init_model = mobilenetv2().to(device)
initial_w = init_model.state_dict()

# CLG_SGD训练
test_acc, train_loss = CLG_SGD(initial_w, global_round, eta, gamma, K, E, M)

# 打印训练过程中的结果
print("CLG_SGD 训练完成！")
print("各轮平均测试精度:", test_acc)
print("各轮平均训练损失:", train_loss)
print("最终测试精度:", test_acc[-1] if len(test_acc) > 0 else "无数据")


init_model = mobilenetv2().to(device)
initial_w = init_model.state_dict()

# Fed_C训练
test_acc, train_loss = Fed_C(initial_w, global_round, eta, gamma, K, E, M)

# 打印训练过程中的结果
print("Fed_C 训练完成！")
print("各轮平均测试精度:", test_acc)
print("各轮平均训练损失:", train_loss)
print("最终测试精度:", test_acc[-1] if len(test_acc) > 0 else "无数据")


init_model = mobilenetv2().to(device)
initial_w = init_model.state_dict()

# Fed_S训练
test_acc, train_loss = Fed_S(initial_w, global_round, eta, gamma, K, E, M)

# 打印训练过程中的结果
print("Fed_S 训练完成！")
print("各轮平均测试精度:", test_acc)
print("各轮平均训练损失:", train_loss)
print("最终测试精度:", test_acc[-1] if len(test_acc) > 0 else "无数据")

  0%|          | 0/100 [00:00<?, ?it/s]

Server 0: 292 14 14 18 16 12 15 12 18 16 12 13 16 14 18 14 14 12 14 13 17


100%|██████████| 100/100 [03:50<00:00,  2.31s/it]


CLG_SGD 训练完成！
各轮平均测试精度: [0.04797614200209952, 0.05672998865399302, 0.0573964582692416, 0.11516386379113254, 0.13954010783512305, 0.13910725804994312, 0.1649174344047876, 0.1338510011064306, 0.1602840667661173, 0.14879318392821456, 0.1809930160821509, 0.16931657525571903, 0.18938481158666545, 0.17060254029477, 0.17814916310121467, 0.17408824343716633, 0.17235222641992645, 0.20073332842105793, 0.19704552013976728, 0.20899020959759235, 0.20332119857131797, 0.21241373311981854, 0.23656580123190551, 0.19599692719872117, 0.21028480907696945, 0.22565285632276313, 0.226353732581085, 0.20787478881888077, 0.18447524478863098, 0.21252573711310316, 0.20899504068051608, 0.22518926234892217, 0.22886573826613982, 0.23815573803002152, 0.21969383629548708, 0.2336884793485085, 0.234786670818517, 0.23081120400325478, 0.2530664147896586, 0.2341060479054067, 0.23155060392584625, 0.23875538383489833, 0.24080624252716892, 0.2484980892243631, 0.2372325387729043, 0.2575496307988656, 0.2777098599863586, 0.25558

100%|██████████| 100/100 [05:43<00:00,  3.44s/it]


Fed_C 训练完成！
各轮平均测试精度: [0.05671737844749976, 0.04986366998366203, 0.0622441171241474, 0.10778741497497275, 0.08034312449456335, 0.10830147892214635, 0.09816418323247657, 0.13692680835157384, 0.141086747661605, 0.1506789090623761, 0.11756928858143885, 0.16979388743789645, 0.131326419970791, 0.15244259787139736, 0.1324923691749675, 0.12370668636779027, 0.1420606720434988, 0.18698623362027547, 0.1695974506694122, 0.15658051144690244, 0.13683453528256037, 0.13276221335699784, 0.18518964707384933, 0.13990309386350813, 0.1876148191928149, 0.19329890197539906, 0.15455555892681966, 0.1490387336242672, 0.15714866116471848, 0.17889041023431018, 0.18139515320650942, 0.19217905025356058, 0.13810662521593717, 0.16366927322152783, 0.1846785336287082, 0.16696558800204644, 0.17459570495542365, 0.2111115025743686, 0.187400756183983, 0.1469645302186722, 0.19454366390372965, 0.1810446109262645, 0.19268435307850393, 0.1882756581263163, 0.1748368731706157, 0.1885642143529156, 0.15434196366492162, 0.16545543

100%|██████████| 100/100 [05:09<00:00,  3.10s/it]

Fed_S 训练完成！
各轮平均测试精度: [0.04318500004254697, 0.0462974212060436, 0.06494970292224893, 0.09428457365272597, 0.13318785897270521, 0.09670388974822577, 0.10735713651865464, 0.14867764930954533, 0.15902854680430267, 0.150199079964891, 0.14708256837219866, 0.16589637586308548, 0.16007832651745532, 0.1716958732654727, 0.17626949591564323, 0.19243550044809857, 0.16294319201083232, 0.1711086686174081, 0.189863080048313, 0.18506419123579973, 0.18540429611577394, 0.19481705372295938, 0.2117507868663644, 0.19706457743618894, 0.19791764600363862, 0.1988649246743738, 0.172816719570858, 0.18981903109962336, 0.21593012174064208, 0.19510231993436455, 0.20194581386590366, 0.23087631172749995, 0.20555408095567654, 0.19214362542982627, 0.2178654081822052, 0.22890352380990708, 0.23089964284002165, 0.22486732761746558, 0.19561732625693204, 0.2194182489154763, 0.23287656625110345, 0.22299216475483752, 0.20461114588929774, 0.22064841562930204, 0.22079018491215835, 0.22028114833832138, 0.18419873258508423, 0.2